# Test preparation

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
from qcodes.utils.validators import Numbers
from qcodes.instrument import Parameter
from tools import *
import tools.project as pro
from random import random, gauss
import numpy as np

## Instruments

In [1]:
from DAC_SP1060 import DAC_SP1060

In [13]:
dac = DAC_SP1060('sp1060','TCPIP0::192.168.31.5::23::SOCKET')

In [ ]:
def dummyset(voltage):
    return voltage


def dummyget():
    return gauss(10,5)


In [ ]:
I_sd = Parameter(name='I_sd', label='SET current', unit='A', set_cmd=None, get_cmd=dummyget)
v_sd = Parameter(name='v_sd', label='SET sd', unit='V', initial_value=0,
                 set_cmd=dummyset, get_cmd=None, vals=Numbers(max_value=1, min_value=-1))
v_g1 = Parameter(name='v_g1', label='Gate 1', unit='V', initial_value=0,
                 set_cmd=dummyset, get_cmd=None, vals=Numbers(max_value=1, min_value=-1))
v_R = Parameter(name = 'v_R', label = 'Resistance vlotage', unit = 'V',
                 set_cmd = dac.ch12.voltage, get_cmd = dac.ch12.voltage, vals = Numbers(max_value=0.01, min_value=-0.01))

In [ ]:
scan_range = np.linspace(-1, 1, 100)

# scan v.1.1 test

In [ ]:
my_scan = Scan(para_meas=I_sd,
               para_scan=v_sd,
               range_scan=scan_range,
               data_manager=my_manager,
               logger=my_logger,
               scaler=1e9)

In [ ]:
my_scan.scan_1d(range_scan_x=0)


# Scan v.1.2 test


In [ ]:
my_scan = Scan(para_meas=I_sd,
               para_scan=[v_sd,v_g1],
               data_manager=my_manager,
               logger=my_logger,
               scaler=1e9)

In [ ]:
my_scan.set_range_1d(scan_range)

In [ ]:
my_scan._ranges

In [ ]:
my_scan.scan_1d(scan_x=1)

In [ ]:
plt.plot(scan_range, my_scan.data[0,:])

In [ ]:
scan_x_range = np.linspace(-0.1, 0.1, 100)
scan_y_range = np.linspace(-0.1, 0.1, 10)

In [ ]:
my_scan.set_range_2d(scan_x_range, scan_y_range)

In [ ]:
my_scan.scan_2d(scan_x=1,scan_y=0)

In [ ]:
my_manager.id

In [ ]:
plt.pcolormesh(my_scan.data[0,:,:])
plt.colorbar()

In [ ]:
arr = np.array(alist, dtype = object)

## Running time analysis

In [ ]:
from line_profiler import LineProfiler

In [ ]:
profile = LineProfiler(my_scan.scan_2d)

In [ ]:
profile.runcall(my_scan.scan_2d, 1, 0)
profile.print_stats()

In [ ]:
profile = LineProfiler(my_scan.scan_1d)
profile.runcall(my_scan.scan_1d, 1)
profile.print_stats()

In [ ]:
import h5py
import time
from collections import Iterable

def acqusition_generator(para_meas):
    for vz in my_scan.para_meas:
        yield vz()
    
def safe(parameter, endpoint):
    if isinstance(parameter, Iterable):
        for element in parameter:
            ramp(element, endpoint)
    else:
        ramp(parameter, endpoint)

def ramp(parameter, endpoint, step=0.01):
    if endpoint > 50 or step == 0:
        pass
    else:
        start_point = parameter()
        if start_point > endpoint:
            step = -step 
        interval = np.arange(start_point, endpoint, step)
        for val in interval:
            parameter(val)
            time.sleep(0.005)
    parameter(endpoint)
def output(parameter, val):
    if isinstance(parameter, Iterable):
        for element in parameter:
            element(val)
    else:
        parameter(val)

def scan_1d(scan_x: int):
    scan_type = 'scan_1d'
    scan_para_list=[scan_x]
    ranges = my_scan._ranges[scan_type]
    scan_para = [my_scan.para_scan[para] for para in scan_para_list]
    if len(ranges) == len(scan_para):
        my_scan.data = np.zeros((len(my_scan.para_meas),) + my_scan.range_scan_dimension(ranges))
    else:
        raise ValueError('scan_para_list is out of range and it does not match the scan range.')
    my_scan.manager.update_run_id()
    data_file = my_scan.manager.create_hdf5_file
    with h5py.File(data_file, 'a') as file:

        dataset = my_scan.scan_dataset(file, ranges)

        start_time = my_scan.scan_start(scan_para, ranges)

#      my_scan.scan_action(range_2d, scan_xy_para, dataset)
        if len(ranges) == 1:
            with ACQTask(acq_name='art',
             acq_channels='Dev1/ai4',
             sample_rate=1e4,m
             memory_size=1000) as daq:
                for idx, vx in enumerate(ranges[0]):
                    output(scan_para[0], vx)
                    time.sleep(my_scan.sleep)
                    for idz, vz in enumerate(my_scan.para_meas):
                        raw_data = daq.read() / my_scan.scaler
                        my_scan.data[idz, idx] = np.average(raw_data)
                        # ------------------------------------------------------------------------------------------------------
                        my_scan.manager.progress_bar(current=my_scan.data[idz, idx],
                                                  idx=idx,
                                                  idz=idz,
                                                  length=len(ranges[0]))
                    # ----------------------------------------------------------------------------------------------------------
                    dataset[:, idx] = my_scan.data[:, idx]
        elif len(ranges) == 2:
#             self.scan_action_2d(ranges, scan_para_list, dataset)
            for idy, vy in enumerate(ranges[1]):
                safe(scan_para[0], ranges[0][0])
                output(scan_para[1], vy)
                for idx, vx in enumerate(ranges[0]):
                    output(scan_para[0], vx)
                    time.sleep(my_scan.sleep)
                    for idz, vz in enumerate(my_scan.para_meas):
                        raw_data = vz() / my_scan.scaler
                        my_scan.data[idz, idx, idy] = np.average(raw_data)
                        my_scan.manager.progress_bar(current=my_scan.data[idz, idx, idy],
                                                  idy=idy,
                                                  idx=idx,
                                                  idz=idz,
                                                  length=len(ranges[0]))

                dataset[:, :, idy] = my_scan.data[:, :, idy]
        else:
            raise ValueError('The dimension of scan ranges excesses two !')

        end_time = my_scan.scan_end(scan_para)

        notes = my_scan.generate_notes(start_time, end_time)

        for key in notes.keys():
            dataset.attrs[key] = notes[key]

    return notes

def scan_2d(scan_x: int, scan_y: int):
    scan_type = 'scan_2d'
    scan_para_list=[scan_x, scan_y]
    ranges = my_scan._ranges[scan_type]
    scan_para = [my_scan.para_scan[para] for para in scan_para_list]
    if len(ranges) == len(scan_para):
        my_scan.data = np.zeros((len(my_scan.para_meas),) + my_scan.range_scan_dimension(ranges))
    else:
        raise ValueError('scan_para_list is out of range and it does not match the scan range.')
    my_scan.manager.update_run_id()
    data_file = my_scan.manager.create_hdf5_file
    with h5py.File(data_file, 'a') as file:

        dataset = my_scan.scan_dataset(file, ranges)

        start_time = my_scan.scan_start(scan_para, ranges)

#         my_scan.scan_action(range_2d, scan_xy_para, dataset)
        if len(ranges) == 1:
#             self.scan_action_1d(ranges, scan_para_list, dataset)
            for idx, vx in enumerate(ranges[0]):
                output(scan_para[0], vx)
                time.sleep(my_scan.sleep)
                for idz, vz in enumerate(my_scan.para_meas):
                    raw_data = vz() / my_scan.scaler
                    my_scan.data[idz, idx] = np.average(raw_data)
                    # ------------------------------------------------------------------------------------------------------
                    my_scan.manager.progress_bar(current=my_scan.data[idz, idx],
                                              idx=idx,
                                              idz=idz,
                                              length=len(ranges[0]))
                # ----------------------------------------------------------------------------------------------------------
                dataset[:, idx] = my_scan.data[:, idx]
        elif len(ranges) == 2:
#             self.scan_action_2d(ranges, scan_para_list, dataset)
            for idy, vy in enumerate(ranges[1]):
                safe(scan_para[0], ranges[0][0])
                output(scan_para[1], vy)
                for idx, vx in enumerate(ranges[0]):
                    output(scan_para[0], vx)
                    time.sleep(my_scan.sleep)
                    for idz, vz in enumerate(my_scan.para_meas):
                        raw_data = vz() / my_scan.scaler
                        my_scan.data[idz, idx, idy] = np.average(raw_data)
                        my_scan.manager.progress_bar(current=my_scan.data[idz, idx, idy],
                                                  idy=idy,
                                                  idx=idx,
                                                  idz=idz,
                                                  length=len(ranges[0]))

                dataset[:, :, idy] = my_scan.data[:, :, idy]
        else:
            raise ValueError('The dimension of scan ranges excesses two !')
            
        end_time = my_scan.scan_end(scan_para)

        notes = my_scan.generate_notes(start_time, end_time)

        for key in notes.keys():
            dataset.attrs[key] = notes[key]

    return notes

In [ ]:
profile = LineProfiler(scan_2d)
profile.runcall(scan_2d, 1, 0)
profile.print_stats()

In [ ]:
class ACQTask:
    def __init__(self, acq_name: str):
        self.acq = acq_name
        self._acq_controller = {'art':self.art, 'm2p':self.m2p}

    def __enter__(self):
#         if self.acq == 'art':
#             print('art acq')
#         elif self.acq == 'm2p':
#             print('m2p acq')
        self._acq_controller[self.acq]()
        return self
    
    @classmethod
    def get_acq(cls, acq_name):
        return getattr(cls,acq_name)
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()
    
    @staticmethod
    def art():
        print('art action')
    
    @staticmethod
    def m2p():
        print('m2p action')
    
    def close(self):
        print('close')

In [ ]:
with ACQTask('m2p') as task:
    print(dir(task))

In [ ]:
task = ACQTask('m2p')

In [ ]:
task.get_acq('m2p')()

# Scan v.1.3 test

In [ ]:
with pro.ACQTask(acq_name='art',
                 acq_channels='Dev1/ai4',
                 sample_rate=1e4,
                 memory_size=1000) as my_task:
    raw_data = my_task.read()


In [ ]:
my_project = Project(sample_name='SAQ14',
                     temperature='mK',
                     tester='XF')

In [ ]:
my_scan = Scan(para_meas=I_sd,
               para_scan=[v_sd, v_g1],
               project=my_project,
               scaler=1e0)

In [ ]:
my_scan.set_range_1d(scan_range=scan_range)

In [ ]:
my_scan.scan_1d(scan_x=0)

In [ ]:
plt.plot(scan_range, my_scan.data[0,:])

In [ ]:
my_scan.scaler

In [ ]:
scan_x_range = np.linspace(-0.1, 0.1, 100)
scan_y_range = np.linspace(-0.1, 0.1, 10)

In [ ]:
my_scan.set_range_2d(scan_x_range, scan_y_range)

In [ ]:
my_scan.scan_2d(scan_x=1,scan_y=0)

In [ ]:
plt.pcolormesh(my_scan.data[0,:,:])
plt.colorbar()

In [ ]:
plt.plot(raw_data)

In [ ]:
np.average(raw_data)

In [ ]:
type(raw_data)

# Scan v.2.1 test